## Data Preparation

You should prepare the following before running this step. Please refer to the `example_data` folder for guidance:

1. **high-resolution CMR contours** from [Cardiac super-resolution label maps](https://data.mendeley.com/datasets/pw87p286yx/1)
   - please download this dataset 
   - we prepare two examples ```example_data/processed_HR_data/1081``` and ```/1256```

---

## Data simulation
Simulate the downsampled and slice-misaligned data from high-resolution CMR contours so that we have paired data for supervised training.

### Docker environment
Please use `docker`, it will build a tensorflow-based container


In [1]:
import os
import sys
sys.path.append('/workspace/Documents')  ### remove this if not needed!
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import glob as gb
import nibabel as nb
import shutil
import os
import math

import CMR_MC_SR_End2End.functions_collection as ff
import CMR_MC_SR_End2End.data_simulation.transformation as transform
import CMR_MC_SR_End2End.Image_utils as util
import CMR_MC_SR_End2End.data_simulation.generate_moving_data as gen

main_path = '/mnt/camca_NAS/CMR_processing/' # replace with your own

### define moving range

In [2]:
# define moving range:
t_mu = 2.5 #unit mm
t_sigma = 0.75 # unit mm
t_bar = 5 # unit mm <= mu + 2sigma ~ 5mm (4 pixels)
extreme = False

### define patient list

In [3]:
patient_list = ff.find_all_target_files(['*'],os.path.join(main_path,'example_data/processed_HR_data'))
case_list = []
for p in patient_list:
    cases = ff.find_all_target_files(['ED'],p)
    for c in cases:
        case_list.append(c)
print(case_list)

main_save_folder = os.path.join(main_path,'example_data/simulated_data')
ff.make_folder([main_save_folder])

['/mnt/camca_NAS/CMR_processing/example_data/processed_HR_data/1081/ED', '/mnt/camca_NAS/CMR_processing/example_data/processed_HR_data/1256/ED']


### do simulation
it will generate two types of data saved in ```example_data/simulated_data```.

- first is downsampled data (downsample a factor of 5 in z-axis), saved in folder ```ds```
- second is applying inter-slice motion to downsampled data, mimicing the motion artifacts, saved in folder ```normal_motion_X```

In [4]:
for case in case_list:
    patient_id = os.path.basename(os.path.dirname(case))
    tf = os.path.basename(case)
    print('patient: ', patient_id, tf)

    save_folder = os.path.join(main_save_folder,patient_id,tf)
    ff.make_folder([os.path.dirname(save_folder),save_folder])


    img_file = os.path.join(case, 'HR_ED_zoomed_crop_flip_clean.nii.gz')
    assert os.path.isfile(img_file) == 1, 'Image file not found: ' + img_file
    img_file = nb.load(img_file)
    affine_HR = img_file.affine
    spacing = img_file.header.get_zooms()
    img = img_file.get_fdata()
    

    # generate static data (only down-sampling)
    img_ds,new_affine = util.downsample_in_z(img,5,affine=affine_HR)
    assert img_ds.shape[-1] == 12
    new_spacing = (spacing[0],spacing[1],spacing[2]*5)      
    new_header = img_file.header
    new_header['pixdim'] = [-1, new_spacing[0], new_spacing[1], new_spacing[-1],0,0,0,0]

    
    save_folder_static = os.path.join(save_folder,'ds'); ff.make_folder([save_folder_static])
    img_ds_nb = nb.Nifti1Image(img_ds, new_affine, header=new_header)
    nb.save(img_ds_nb, os.path.join(save_folder_static,'data_clean.nii.gz'))
        

    # generate moving data 
    for i in range(0,2):

        print('generate normal motion', i+1)
        save_folder_r = os.path.join(save_folder,'normal_motion_'+str(i+1)); ff.make_folder([save_folder_r])

        img_new,record = gen.generate_moving_data(img_ds, t_mu, t_sigma, t_bar, 0, spacing, order = 0, extreme = extreme)
        # save image
        nb.save(nb.Nifti1Image(img_new, new_affine, header=new_header), os.path.join(save_folder_r,'data_clean.nii.gz'))
        # save record
        ff.txt_writer2(os.path.join(save_folder_r, 'motion_record.txt'),record)
        np.save(os.path.join(save_folder_r, 'motion_record.npy'), np.asarray(record))
    



patient:  1081 ED
generate normal motion 1
generate normal motion 2
patient:  1256 ED
generate normal motion 1
generate normal motion 2


/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


### calculate the centerline points (will be used in the following steps)

In [5]:
# get patient list
case_list = ff.find_all_target_files(['*/ED'],os.path.join(main_path,'example_data/simulated_data'))
print(case_list)

# get center line points (for LV + myo)
for case in case_list:
    patient_id = os.path.basename(os.path.dirname(case))
    tf = os.path.basename(case)

    files = ff.find_all_target_files(['ds/data_clean.nii.gz', 'normal_motion*/data_clean.nii.gz'], case)

    for img_file in files:
        print(img_file)
        img = nb.load(img_file).get_fdata()
        img= np.round(img)
        img = img.astype(int)
        img = util.relabel(img,4,0)

        # find centerline points
        slice_list = []; center_list_raw = []
        for i in range(0,img.shape[-1]):
            I = img[:,:,i]
            # no heart:
            if np.where(I > 0)[0].shape[0] < 20 :
                center_list_raw.append(util.center_of_mass(np.zeros((20,20)),0,large = True))
                continue
            slice_list.append(i)
            center_list_raw.append(np.round(util.center_of_mass(I,0,large = True),2))

        np.save(os.path.join(os.path.dirname(img_file), 'centerlist.npy'), np.asarray(center_list_raw))
        ff.txt_writer(os.path.join(os.path.dirname(img_file),'centerlist.txt'), center_list_raw, ['']*len(center_list_raw))

['/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1081/ED'
 '/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1256/ED']
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1081/ED/ds/data_clean.nii.gz
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1081/ED/normal_motion_1/data_clean.nii.gz
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1081/ED/normal_motion_2/data_clean.nii.gz
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1256/ED/ds/data_clean.nii.gz
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1256/ED/normal_motion_1/data_clean.nii.gz
/mnt/camca_NAS/CMR_processing/example_data/simulated_data/1256/ED/normal_motion_2/data_clean.nii.gz


/usr/local/lib/python3.8/dist-packages/scipy/ndimage/measurements.py:1406: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer
